# Advanced RAG Exercise

This notebook is designed as an exercise to build a complete Retrieval-Augmented Generation (RAG) system. In this exercise, you will integrate three main components into a single pipeline:

1. **Retrieval Module** – Retrieve relevant documents based on a query.
2. **Transformation Module** – Transform the retrieved queries.
3. **Generation Module and Evaluation** – Use the transformed data to generate responses and evaluate the overall system performance.

In [1]:
import tqdm
import glob
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings  # For generating embeddings for text chunks
import faiss
import pickle
from dotenv import load_dotenv
import os
from groq import Groq
from sentence_transformers import SentenceTransformer
import random
from sentence_transformers import CrossEncoder
import numpy as np


## 1. Building the RAG Pipeline

Load the data and store it in a string.

In [2]:
# 📚 Importiere benötigte Bibliotheken
from PyPDF2 import PdfReader          # Zum Lesen von PDF-Dateien
import glob                           # Zum Finden von Dateien basierend auf einem Pfad-Muster
import tqdm                           # Für eine visuelle Fortschrittsanzeige beim Durchlaufen der Dateien

# 📂 Definiere den Pfad zu allen PDF-Dateien im Verzeichnis "data/"
# Das Sternchen (*) bedeutet: alle Dateien mit der Endung .pdf
glob_path = "data/*.pdf"

# 📝 Lege einen leeren String an, um den extrahierten Text aller PDFs zu sammeln
text = ""

# 🔁 Durchlaufe alle PDF-Dateien im angegebenen Pfad mit Fortschrittsanzeige
for pdf_path in tqdm.tqdm(glob.glob(glob_path)):

    # 📄 Öffne die aktuelle PDF-Datei im Lese-/Binärmodus
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)  # Erstelle ein PDF-Reader-Objekt

        # 🧾 Extrahiere den Text von allen Seiten, falls Text vorhanden ist
        text += " ".join(
            page.extract_text()           # Text extrahieren
            for page in reader.pages      # Für jede Seite im PDF
            if page.extract_text()        # Nur wenn Text vorhanden ist (keine leere Seite)
        )

# 👁️ Zeige die ersten 50 Zeichen des zusammengefügten Textes als Vorschau
text[:50]


100%|██████████| 2/2 [00:03<00:00,  1.74s/it]


'Asthma: diagnosis, \nmoni toring and chr onic \nasth'

Split the data into chunks.

In [3]:
# 📦 Importiere den Textsplitter (falls noch nicht geschehen)
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔧 Erstelle einen Text-Splitter mit folgenden Parametern:
# - chunk_size: max. 2000 Zeichen pro Chunk
# - chunk_overlap: jeweils 200 Zeichen Überlappung zwischen zwei Chunks
#   → sorgt dafür, dass der Kontext beim Übergang zwischen Chunks nicht verloren geht
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,         # Maximale Länge eines Chunks in Zeichen
    chunk_overlap=200        # Überlappung zwischen benachbarten Chunks
)

# ✂️ Teile den extrahierten PDF-Text mithilfe des Splitters in kleinere, überlappende Textabschnitte
chunks = splitter.split_text(text)

# 🧾 Jetzt enthält die Variable 'chunks' eine Liste von Textabschnitten, die jeweils max. 2000 Zeichen lang sind


In [4]:
# 📊 Zeige die Gesamtanzahl der erzeugten Text-Chunks an
# Das ist hilfreich zur Kontrolle, wie viele Abschnitte aus dem PDF-Text entstanden sind
print(f"Total chunks: {len(chunks)}")

# 👁️‍🗨️ Zeige eine Vorschau auf den ersten Chunk (die ersten 200 Zeichen)
# So kannst du kontrollieren, ob die Aufteilung sinnvoll funktioniert hat
print("Preview of the first chunk:", chunks[0][:200])


Total chunks: 130
Preview of the first chunk: Asthma: diagnosis, 
moni toring and chr onic 
asthma manag emen t (BTS, 
NICE, SI GN) 
NICE guideline 
Published: 27 No vember 202 4 
www .nice.or g.uk/guidance/ng2 45 
© NICE 202 4. All right s reser


## Choose an embedding model
Use the SentenceTransfomer wrapper as we have done so far.
Models are found here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
or on HuggingFace.

Embed the chunks.

In [5]:
# Definiere den Namen des Embedding-Modells, das verwendet werden soll.
# Dieses Modell wurde trainiert, um semantisch ähnliche Sätze in ähnliche Vektoren zu übersetzen.
# "multilingual" bedeutet, dass es mit mehreren Sprachen (z. B. Englisch, Deutsch) umgehen kann.
model_name = "paraphrase-multilingual-MiniLM-L12-v2"

# Lade das ausgewählte Modell mit der SentenceTransformer-Bibliothek.
# Es wird intern von HuggingFace geladen und kann sofort zur Vektorisierung verwendet werden.
model = SentenceTransformer(model_name)

# Erzeuge Embeddings (Vektoren) für alle Text-Chunks.
# convert_to_numpy=True sorgt dafür, dass du ein NumPy-Array zurückbekommst (praktisch für spätere Verarbeitung).
chunk_embeddings = model.encode(chunks, convert_to_numpy=True)


## 3. Build Index and save index

In [6]:
# 📐 Ermittle die Anzahl der Dimensionen eines einzelnen Embedding-Vektors
# chunk_embeddings ist ein 2D-Array mit der Form (Anzahl der Chunks, Anzahl der Dimensionen)
# z. B. (120, 384) → 120 Chunks, jeder als Vektor mit 384 Werten

d = chunk_embeddings.shape[1]  # Index [1] gibt die Spaltenanzahl = Vektor-Dimension

# 🖨️ Gib die Dimension des Embeddings aus (wichtig für FAISS oder Ähnlichkeitsvergleiche)
print(d)


384


🔢 Was bedeutet das Ergebnis 384?
Das Ergebnis 384 bedeutet, dass jedes deiner Chunks durch das Embedding-Modell in einen Vektor mit 384 Dimensionen umgewandelt wurde.

🔍 Was heißt das konkret?
Du hattest eine Liste mit vielen Chunks, z. B.:

python
Kopieren
Bearbeiten
chunks = ["Textabschnitt 1", "Textabschnitt 2", ...]
Mit dem Embedding-Modell:

python
Kopieren
Bearbeiten
chunk_embeddings = model.encode(chunks, convert_to_numpy=True)
wurde jeder Chunk in einen Vektor umgerechnet.

Jeder dieser Vektoren enthält 384 Zahlenwerte, z. B.:

csharp
Kopieren
Bearbeiten
[0.12, -0.34, 0.87, ..., 0.01]  →  Länge: 384
💡 Warum genau 384?
Das liegt am verwendeten Modell:

paraphrase-multilingual-MiniLM-L12-v2 erzeugt standardmäßig 384-dimensionale Embeddings.

🔁 Andere Modelle liefern z. B.:

768 (z. B. BERT-Base)

1024 (z. B. RoBERTa-Large)

📌 Bedeutung in der Praxis
Du kannst dir jeden Vektor als Punkt in einem 384-dimensionalen Raum vorstellen.

Zwei Chunks, die semantisch ähnlich sind, liegen nahe beieinander in diesem Raum.

Das bildet die Grundlage für:

✅ semantische Suche

✅ Clustering

✅ Klassifikation

✅ Ähnlichkeitsvergleiche

🧠 Wichtig für FAISS
Wenn du z. B. mit FAISS einen Vektorindex aufbaust, musst du die Vektordimension angeben. In deinem Fall ist das:

python
Kopieren
Bearbeiten
d = 384
FAISS erwartet diese Angabe, um korrekt mit den Vektoren arbeiten zu können.

Möchtest du auch ein kurzes Beispiel mit np.linalg.norm() oder Cosinus-Ähnlichkeit zur Visualisierung von Vektorabstand?

In [7]:
# 📦 Erstelle einen FAISS-Index zur schnellen Ähnlichkeitssuche
# Wir verwenden hier "IndexFlatL2", der auf der euklidischen Distanz (L2) basiert.
# Der Parameter `d` gibt die Anzahl der Dimensionen pro Vektor an (z. B. 384 bei deinem Modell).
index = faiss.IndexFlatL2(d)

# ➕ Füge alle zuvor erzeugten Embedding-Vektoren in den Index ein
# Dadurch kann FAISS später Anfragen (Queries) mit diesen vergleichen
index.add(chunk_embeddings)

# 🔢 Gib aus, wie viele Vektoren im Index gespeichert sind
# Sollte gleich der Anzahl deiner Chunks sein
print("Number of embeddings in FAISS index:", index.ntotal)


Number of embeddings in FAISS index: 130


In [8]:
# 💾 Speichere den FAISS-Index auf der Festplatte
# → Damit musst du den Index beim nächsten Mal nicht neu berechnen
#    (spart Zeit beim späteren Wiederverwenden)
faiss.write_index(index, "faiss/faiss_index.index")

# 🗂️ Speichere zusätzlich die Text-Chunks als Mapping (Index → Originaltext)
# → So kannst du später zu jedem Treffer die zugehörige Textpassage finden
with open("faiss/chunks_mapping.pkl", "wb") as f:
    pickle.dump(chunks, f)  # Serialisiere und speichere die Liste der Chunks


## Load Key for language Models

In [39]:
# 🔄 Lade Umgebungsvariablen aus einer .env-Datei (falls vorhanden)
# Die Funktion `load_dotenv()` sucht nach einer Datei namens `.env` im Projektverzeichnis
# und lädt alle darin enthaltenen Variablen als Umgebungsvariablen (environment variables).
load_dotenv()

# 🔐 Lies den Google-API-Schlüssel aus der Umgebungsvariable GOOGLE_API_KEY
# Wenn die Variable nicht gesetzt ist, wird `None` zurückgegeben – es erfolgt KEIN Fehler.
google_api_key = os.getenv("GOOGLE_API_KEY")

# 🔐 Lies den OpenAI-API-Schlüssel aus der Umgebungsvariable OPENAI_API_KEY
# Auch hier: kein Fehler, falls der Schlüssel nicht vorhanden ist.
openai_api_key = os.getenv("OPENAI_API_KEY")

# 🔐 Lies den Groq-API-Schlüssel aus der Umgebungsvariable GROQ_API_KEY
# Der Rückgabewert ist `None`, falls keine solche Umgebungsvariable existiert.
groq_api_key = os.getenv("GROQ_API_KEY")

# 💡 Verhalten:
# - Wenn keine `.env`-Datei vorhanden ist, oder eine Variable fehlt,
#   dann sind die entsprechenden Variablen (`google_api_key`, etc.) = `None`.
# - Das löst an dieser Stelle **noch keinen Fehler aus**.
# - Fehler treten erst auf, wenn du z. B. versuchst, einen LLM-Client mit einem `None`-Key zu initialisieren.
#
# ✅ Optionaler Schutz (empfohlen):
# assert google_api_key is not None, "GOOGLE_API_KEY not found!"


## 4. Build a retriever function

arguments: query, k, index, chunks, embedding model

return: retrieved texts, distances

In [40]:
def retrieve_texts(query, k, index, token_split_texts, model):
    """
    🔎 Funktion zur semantischen Suche:
    Gibt die top-k ähnlichsten Text-Chunks (und ihre Distanzen) zu einer gegebenen Anfrage (query) zurück.

    Parameter:
    - query: Die vom Benutzer gestellte Suchfrage (Textstring)
    - k: Anzahl der gewünschten ähnlichsten Treffer (z. B. 3)
    - index: FAISS-Index mit gespeicherten Embeddings der Chunks
    - token_split_texts: Die Original-Textchunks (z. B. nach Tokenisierung)
    - model: Das Embedding-Modell zur Umwandlung der Anfrage in einen Vektor

    Rückgabe:
    - retrieved_texts: Liste der k ähnlichsten Textabschnitte (aus den Chunks)
    - distances: Liste der zugehörigen Distanzen (kleiner = ähnlicher)
    """
    
    # 📐 Wandelt die Textanfrage in einen Embedding-Vektor um
    # `convert_to_numpy=True` liefert ein NumPy-Array, wie es FAISS erwartet
    query_embedding = model.encode([query], convert_to_numpy=True)
    
    # 🔍 Durchsuche den FAISS-Index mit dem erzeugten Vektor
    # Gibt die k ähnlichsten Vektoren zurück (nach euklidischer Distanz)
    distances, indices = index.search(query_embedding, k)
    
    # 📋 Extrahiere die zugehörigen Original-Text-Chunks basierend auf den gefundenen Indizes
    retrieved_texts = [token_split_texts[i] for i in indices[0]]
    
    # 🔁 Gib die Treffer und ihre Distanzen zurück
    return retrieved_texts, distances


## 5. Build an answer function
Build an answer function that takes a query, k, an index and the chunks.

return: answer

In [41]:
def answer_query(query, k, index, texts):
    """
    📥 Diese Funktion beantwortet eine Benutzerfrage basierend auf den k ähnlichsten Chunks.
    Sie verwendet semantische Suche (FAISS), um relevante Kontexte zu finden,
    und ein LLM (über Groq), um daraus eine Antwort zu generieren.
    
    Parameter:
    - query (str): Die Benutzerfrage.
    - k (int): Anzahl der zu verwendenden ähnlichsten Chunks.
    - index: FAISS-Index mit Embeddings.
    - texts: Die ursprünglichen Chunks (z. B. aus PDF-Texten).

    Rückgabe:
    - answer (str): Die generierte Antwort des LLM.
    """

    # 🧠 Initialisiere das Embedding-Modell (gleich wie beim Erstellen des FAISS-Index)
    model_name = "paraphrase-multilingual-MiniLM-L12-v2"
    model = SentenceTransformer(model_name)

    # 🔍 Verwende die Retriever-Funktion, um die k ähnlichsten Chunks zur query zu finden
    retrieved_texts, _ = retrieve_texts(query, k, index, texts, model)

    # 📚 Fasse die gefundenen Text-Chunks zu einem Kontextblock zusammen
    context = "\n\n".join(retrieved_texts)

    # 🧾 Baue den Prompt: Erkläre die Antwort kindgerecht auf Basis des Kontexts
    prompt = (
        "Answer the following question using the provided context. "
        "Explain it as if you are explaining it to a 5 year old.\n\n"
        "Context:\n" + context + "\n\n"
        "Question: " + query + "\n"
        "Answer:"
    )

    # 🔐 Initialisiere den Groq-Client mit dem API-Key
    client = Groq(api_key=groq_api_key)

    # 💬 Baue die Nachricht im OpenAI-kompatiblen Format
    messages = [
        {
            "role": "system",  # Rolle: System gibt Instruktionen vor
            "content": prompt  # Prompt mit Kontext + Frage
        }
    ]

    # 🤖 Anfrage an das LLM (hier: llama3-70b über Groq)
    llm = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )

    # 📤 Extrahiere und gib die Antwort zurück
    answer = llm.choices[0].message.content
    return answer


#### Test your RAG

In [42]:
# ❓ Definiere die Benutzerfrage (Query), die beantwortet werden soll
query = "What is the most important factor in diagnosing asthma?"

# 🤖 Rufe die Antwortfunktion auf, um die Query durch semantische Suche + LLM beantworten zu lassen
# - Nutzt die top-5 ähnlichsten Chunks (k = 5)
# - Verwendet den FAISS-Index (index) und die Original-Chunks (chunks)
answer = answer_query(query, 5, index, chunks)

# 📤 Gib die generierte Antwort aus
print("LLM Answer:", answer)


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

## 6. Create a Rewriter

Take a query and an api key for the model and rewrite the query. 

Rewriting a query: A Language Model is prompted to rewrite a query to better suit a task.

Other Transfomrations are implemented in a similar fashion, this is just an example!

In [18]:
def rewrite_query(query, groq_api_key):
    """
    ✍️ Rewrite the user's query to better suit medical guideline retrieval.

    Ziel:
    - Formuliere die ursprüngliche Anfrage so um, dass sie präziser und besser für die Suche in medizinischen Richtlinien geeignet ist.

    Parameter:
    - query (str): Die originale Benutzeranfrage (z. B. "Tell me more about asthma").
    - groq_api_key (str): Dein Groq API Key zur Authentifizierung des LLM-Zugriffs.

    Rückgabe:
    - rewritten_query (str): Die umformulierte Anfrage, angepasst an den medizinischen Kontext.
    """

    # 🧠 Initialisiere den Groq-Client mit deinem API-Schlüssel
    client = Groq(api_key=groq_api_key)

    # 🧾 Prompt für das Sprachmodell, um die Anfrage umzuschreiben
    rewriting_prompt = (
        "Rewrite the following query into a format, such that it can be answered by looking at medical guidelines. "
        "Keep the keywords but ensure that it is close to a format, such as in medical guidelines. "
        "Just answer with the rewritten query\n\n"
        "Query: " + query
    )

    # 📨 Baue die Nachrichtenstruktur für das Chat-Modell (nur eine Systemnachricht mit dem Prompt)
    messages = [
        {"role": "system", "content": rewriting_prompt}
    ]

    # 🤖 Sende die Anfrage an das LLM (hier: llama-3.3-70b-versatile) und erhalte die Antwort
    llm = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile",
    )

    # ✂️ Extrahiere die Antwort aus der LLM-Antwortstruktur und entferne Whitespace
    rewritten_query = llm.choices[0].message.content.strip()

    # 🔙 Gib die umgeschriebene Anfrage zurück
    return rewritten_query


## 7. Implement the rewriter into your answer function

In [20]:
def answer_query_with_rewriting(query, k, index, texts, groq_api_key):
    """
    ❓ Funktion zur Beantwortung einer Nutzeranfrage mit Rewriting und semantischer Suche.
    Diese Version verwendet zusätzlich eine Rewriter-Funktion, um die ursprüngliche Frage
    zu optimieren, bevor sie verarbeitet wird.

    Parameter:
    - query (str): Die ursprüngliche Frage des Benutzers.
    - k (int): Anzahl der zurückzugebenden ähnlichsten Text-Chunks.
    - index: FAISS-Index, der alle gespeicherten Embeddings enthält.
    - texts (list): Liste der ursprünglichen Chunks (Textabschnitte).
    - groq_api_key (str): API-Schlüssel für das Groq-LLM.

    Rückgabe:
    - answer (str): Die vom Language Model generierte Antwort.
    """

    # 🧠 Lade das Embedding-Modell, um Abfragen in Vektoren umzuwandeln
    model_name = "paraphrase-multilingual-MiniLM-L12-v2"
    model = SentenceTransformer(model_name)

    # ✏️ Wende den Rewriter auf die ursprüngliche Frage an,
    # um sie besser für die semantische Suche geeignet zu machen
    rewritten_query = rewrite_query(query, groq_api_key)    
    print("Rewritten Query:", rewritten_query)  # Debug-Ausgabe für die überarbeitete Frage

    # 🔍 Rufe die semantisch ähnlichsten Text-Chunks ab (auf Basis der rewritten_query)
    retrieved_texts, _ = retrieve_texts(rewritten_query, k, index, texts, model)

    # 📚 Kombiniere die abgerufenen Chunks zu einem Kontext-Block
    context = "\n\n".join(retrieved_texts)

    # 📝 Erstelle einen Prompt, der den Kontext und die ursprüngliche Frage enthält
    # und das Modell auffordert, die Antwort kindgerecht zu formulieren
    prompt = (
        "Answer the following question using the provided context. "
        "Explain it as if you are explaining it to a 5 year old.\n\n"
        "Context:\n" + context + "\n\n"
        "Question: " + query + "\n"
        "Answer:"
    )

    # 🤖 Initialisiere den Groq-Client mit deinem API-Key
    client = Groq(api_key=groq_api_key)

    # 💬 Formuliere die Nachricht für das LLM im System-Kontext
    messages = [
        {"role": "system", "content": prompt}
    ]

    # 🧠 Verwende das LLM (hier: llama-3.3-70b-versatile) zur Generierung der Antwort
    llm = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )

    # 📤 Extrahiere und gib die generierte Antwort zurück
    answer = llm.choices[0].message.content
    return answer


#### Test it

In [22]:
# 🧾 Definiere die ursprüngliche Nutzerfrage
# Diese Frage wird später durch die Rewriter-Funktion überarbeitet
query = "What is the most important factor in diagnosing asthma?"

# 🧠 Rufe die Hauptfunktion zur Beantwortung der Frage auf
# Diese Funktion:
# - rewritten die Frage mit Hilfe eines LLMs
# - sucht die top 5 (k=5) ähnlichsten Chunks aus dem FAISS-Index
# - erstellt daraus einen Kontext
# - und ruft dann ein Language Model (LLM) auf, um basierend auf diesem Kontext eine Antwort zu generieren
answer = answer_query_with_rewriting(query, 5, index, chunks, groq_api_key)

# 🖨️ Gib die Antwort des LLMs im Notebook aus
print("LLM Answer:", answer)


Rewritten Query: What are the diagnostic criteria for asthma, and which clinical feature or symptom is considered the primary indicator for a definitive diagnosis according to current clinical guidelines?
LLM Answer: Oh boy, let's talk about asthma. So, you know how sometimes people have trouble breathing and it can be really scary? Well, doctors need to figure out if someone has asthma or not. 

The most important thing in diagnosing asthma is looking at how someone's body reacts when they breathe. Doctors can do special tests like spirometry, FeNO measurements, or skin prick tests to see how their lungs work. They can also check for something called IgE in their blood, which can help them understand if someone has allergies that might be making their asthma worse.

But, the best way to know for sure if someone has asthma is to do a special test called a bronchial challenge test. This test is like a big puzzle that helps doctors understand how someone's lungs react to different things

## 8 .Evaluation

Select random chunks from all your chunks, and generate a question to each of these chunks

In [23]:
import time
import httpx  # Ensure you're catching the correct timeout exception
from openai import OpenAI
def generate_questions_for_random_chunks(chunks, num_chunks=20, max_retries=3):
    """
    Randomly selects a specified number of text chunks from the provided list,
    then generates a question for each selected chunk using the Groq LLM.

    Parameters:
    - chunks (list): List of text chunks.
    - groq_api_key (str): Your Groq API key.
    - num_chunks (int): Number of chunks to select randomly (default is 20).

    Returns:
    - questions (list of tuples): Each tuple contains (chunk, generated_question).
    """
    # Randomly select the desired number of chunks.
    selected_chunks = random.sample(chunks, num_chunks)
    
    # Initialize the Groq client once
    client = OpenAI(api_key=openai_api_key)
    
    questions = []
    for chunk in tqdm.tqdm(selected_chunks):
        # Build a prompt that asks the LLM to generate a question based on the chunk.
        prompt = (
            "Based on the following text, generate an insightful question that covers its key content:\n\n"
            "Text:\n" + chunk + "\n\n"
            "Question:"
        )
        
        messages = [
            {"role": "system", "content": prompt}
        ]
        
        generated_question = None
        attempt = 0
        
        # Try calling the API with simple retry logic.
        while attempt < max_retries:
            try:
                llm_response = client.chat.completions.create(
                     model="gpt-4o-mini",
                    messages=messages
                )
                generated_question = llm_response.choices[0].message.content.strip()
                break  # Exit the loop if successful.
            except httpx.ReadTimeout:
                attempt += 1
                print(f"Timeout occurred for chunk. Retrying attempt {attempt}/{max_retries}...")
                time.sleep(2)  # Wait a bit before retrying.
        
        # If all attempts fail, use an error message as the generated question.
        if generated_question is None:
            generated_question = "Error: Failed to generate question after several retries."
        
        questions.append((chunk, generated_question))
    
    return questions

#### Test it

In [24]:
questions = generate_questions_for_random_chunks(chunks, num_chunks=5, max_retries=2)
for idx, (chunk, question) in enumerate(questions, start=1):
    print(f"Chunk {idx}:\n{chunk[:100]}...\nGenerated Question: {question}\n")

100%|██████████| 5/5 [00:05<00:00,  1.20s/it]

Chunk 1:
52 • Follow the recommendations f or disease-specific indications in t he NICE 
guideline on t heir ...
Generated Question: What specific antihypertensive drug treatments does the NICE guideline recommend for adults with type 2 diabetes or those under 55, and how do these recommendations vary based on genetic background?

Chunk 2:
conditions such as f ood aller gies. 
Bronchial challeng e test 
A test t o measur e airway responsi...
Generated Question: What are the various tests and measures used to assess airway responsiveness and inflammation in asthma, and what do they indicate about the condition?

Chunk 3:
developed t o suppor t healt hcare professionals and people wit h hyper tension t o discuss 
their t...
Generated Question: What factors contribute to the committee's decision to retain and update the existing treatment recommendations for step 4 hypertension management, despite the absence of new evidence?

Chunk 4:
more of t he following: 
• target or gan damage 
• es

## 9.Test the questions with your built retriever

In [25]:
def answer_generated_questions(question_tuples, k, index, texts, groq_api_key):
    """
    For each (chunk, generated_question) tuple in the provided list, use the prebuilt
    retrieval function to generate an answer for the generated question. The function
    returns a list of dictionaries containing the original chunk, the generated question,
    and the answer.
    
    Parameters:
    - question_tuples (list of tuples): Each tuple is (chunk, generated_question)
    - k (int): Number of retrieved documents to use for answering.
    - index: The FAISS index.
    - texts (list): The tokenized text chunks mapping.
    - groq_api_key (str): Your Groq API key.
    
    Returns:
    - results (list of dict): Each dict contains 'chunk', 'question', and 'answer'.
    """
    results = []
    for chunk, question in question_tuples:
        # Use your retrieval-based answer function. Here we assume the function signature is:
        # answer_query(query, k, index, texts, groq_api_key)
        answer = answer_query(question, k, index, texts) #query, k, index,texts
        results.append({
            "chunk": chunk,
            "question": question,
            "answer": answer
        })
    return results

#### Check the results

In [26]:
results = answer_generated_questions(questions, 5, index, chunks, groq_api_key)

for item in results:
    print("Chunk Preview:", item['chunk'][:100])
    print("Generated Question:", item['question'])
    print("Answer:", item['answer'])
    print("-----------------------------")

Chunk Preview: 52 • Follow the recommendations f or disease-specific indications in t he NICE 
guideline on t heir 
Generated Question: What specific antihypertensive drug treatments does the NICE guideline recommend for adults with type 2 diabetes or those under 55, and how do these recommendations vary based on genetic background?
Answer: So, you know how sometimes we talk about medicine and doctors? Well, there's a special group called NICE that helps doctors know what medicines to give to people who are sick. 

For people with high blood pressure, NICE says that if you have type 2 diabetes, or if you're under 55 years old, they recommend a special kind of medicine called an ACE inhibitor or an ARB. It's like a special kind of helper that makes your blood pressure go down.

But, if you're from a family that came from Black Africa or the Caribbean, the doctors might choose a different medicine for you. That's because some medicines work better for some people than others, depending o

## Evaluate the answers

In [27]:
import pandas as pd
def evaluate_answers_binary(results, groq_api_key, max_retries=3):
    """
    Evaluates each answer in the results list using an LLM.
    For each result (a dictionary containing 'chunk', 'question', and 'answer'),
    it sends an evaluation prompt to the Groq LLM which outputs 1 if the answer is on point,
    and 0 if it is missing the point.
    
    Parameters:
    - results (list of dict): Each dict must contain keys 'chunk', 'question', and 'answer'.
    - groq_api_key (str): Your Groq API key.
    - max_retries (int): Maximum number of retries if the API call times out.
    
    Returns:
    - df (pandas.DataFrame): A dataframe containing the original chunk, question, answer, and evaluation score.
    """
    evaluations = []
    client = OpenAI(api_key=openai_api_key)
    
    for item in tqdm.tqdm(results, desc="Evaluating Answers"):
        # Build the evaluation prompt.
        prompt = (
            "Evaluate the following answer to the given question. "
            "If the answer is accurate and complete, reply with 1. "
            "If the answer is inaccurate, incomplete, or otherwise not acceptable, reply with 0. "
            "Do not include any extra text.\n\n"
            "Question: " + item['question'] + "\n\n"
            "Answer: " + item['answer'] + "\n\n"
            "Context (original chunk): " + item['chunk'] + "\n\n"
            "Evaluation (1 for good, 0 for bad):"
        )
        
        messages = [{"role": "system", "content": prompt}]
        
        generated_eval = None
        attempt = 0
        
        # Retry logic in case of timeouts or errors.
        while attempt < max_retries:
            try:
                llm_response = client.chat.completions.create(
                    messages=messages,
                    model="4o-mini"
                )
                generated_eval = llm_response.choices[0].message.content.strip()
                break  # Exit the retry loop if successful.
            except httpx.ReadTimeout:
                attempt += 1
                print(f"Timeout occurred during evaluation. Retrying attempt {attempt}/{max_retries}...")
                time.sleep(2)
            except Exception as e:
                attempt += 1
                print(f"Error during evaluation: {e}. Retrying attempt {attempt}/{max_retries}...")
                time.sleep(2)
        
        # If no valid evaluation was produced, default to 0.
        if generated_eval is None:
            generated_eval = "0"
        
        # Convert the response to an integer (1 or 0).
        try:
            score = int(generated_eval)
            if score not in [0, 1]:
                score = 0
        except:
            score = 0
        
        evaluations.append(score)
    
    # Add the evaluation score to each result.
    for i, item in enumerate(results):
        item['evaluation'] = evaluations[i]
    
    # Create a dataframe for manual review.
    df = pd.DataFrame(results)
    return df

### Display them

In [29]:
df_evaluations = evaluate_answers_binary(results, openai_api_key)
display(df_evaluations)

Evaluating Answers:   0%|          | 0/5 [00:00<?, ?it/s]

Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 1/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 2/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 3/3...


Evaluating Answers:  20%|██        | 1/5 [00:06<00:26,  6.65s/it]

Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 1/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 2/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 3/3...


Evaluating Answers:  40%|████      | 2/5 [00:13<00:19,  6.61s/it]

...

Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 1/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 2/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 3/3...


Evaluating Answers:  80%|████████  | 4/5 [00:26<00:06,  6.57s/it]

Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 1/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 2/3...
Error during evaluation: Error code: 404 - {'error': {'message': 'The model `4o-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. Retrying attempt 3/3...


Evaluating Answers: 100%|██████████| 5/5 [00:32<00:00,  6.58s/it]


,chunk,question,answer,evaluation
0,52 • Follow the recommendations f or disease-s...,What specific antihypertensive drug treatments...,"So, you know how sometimes we talk about medic...",0
1,conditions such as f ood aller gies. \nBronchi...,What are the various tests and measures used t...,"Oh boy, let's talk about asthma tests. So, you...",0
2,developed t o suppor t healt hcare professiona...,What factors contribute to the committee's dec...,"Oh boy, let's talk about something important f...",0
3,more of t he following: \n• target or gan dama...,What role does clinical judgment play in the t...,"So, you know how sometimes we get sick or feel...",0
4,BTS ISBN: 9 78-1-917 619-01-1 \nNICE ISBN: 9 7...,What are the main responsibilities of healthca...,"Oh boy, let's talk about taking care of people...",0
